In [ ]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve

In [ ]:
def mesh_fem_1d(a,b,M,k):
    nrNodes = k*M + 1
    c4n = np.linspace(a, b, nrNodes)
    n4e = np.array([[i*k, (i+1)*k] for i in range(M)])
    n4db = np.array([0, nrNodes-1])
    ind4e = np.array([list(range(i*k, (i+1)*k+1)) for i in range(M)])
    return (c4n,n4e,n4db,ind4e)

**[Exercise 1]** Add the matrices for the cubic approximations ($k=3$) in **get_matrices_1d**

In [ ]:
def get_matrices_1d(k=1):
    if k == 1:
        M_R = np.array([[2, 1],[1, 2]], dtype=np.float64) / 3.
        S_R = np.array([[1, -1],[-1, 1]], dtype=np.float64) / 2.
        D_R = np.array([[-1, 1],[-1, 1]], dtype=np.float64) / 2.
    elif k == 2:
        M_R = np.array([[4, 2, -1],[2, 16, 2],[-1, 2, 4]], dtype=np.float64) / 15.
        S_R = np.array([[7, -8, 1],[-8, 16, -8],[1, -8, 7]], dtype=np.float64) / 6.
        D_R = np.array([[-3, 4, -1],[-1, 0, 1],[1, -4, 3]], dtype=np.float64) / 2.
    elif k == 3:
        M_R = np.array([[128, 99, -36, 19], [99, 648, -81, -36],
            [-36, -81, 648, 99], [19, -36, 99, 128]], dtype=np.float64) / 840.
        S_R = np.array([[148, -189, 54, -13], [-189, 432, -297, 54],
            [54, -297, 432, -189], [-13, 54, -189, 148]], dtype=np.float64) / 80.
        D_R = np.array([[-11, 18, -9, 2], [-2, -3, 6, -1],
            [1, -6, 3, 2], [-2, 9, -18, 11]], dtype=np.float64) / 4.
        
    ###################################################################################################
	# TODO: Add the matrices for the cubic approximations ($k=3$).
	
	###################################################################################################	
    else:
        M_R = 0
        S_R = 0
        D_R = 0
    return (M_R, S_R, D_R)

In [ ]:
def fem_for_poisson_1d(c4n,n4e,n4db,ind4e,k,M_R,S_R,f,u_D):
    number_of_nodes = len(c4n)
    number_of_elements = len(n4e)
    b = np.zeros(number_of_nodes)
    u = np.zeros(number_of_nodes)
    J = np.array([(c4n[n4e[i,1]]-c4n[n4e[i,0]])/2 for i in range(number_of_elements)])
    Aloc = np.array([S_R.flatten()/J[i] for i in range(number_of_elements)])
    for i in range(number_of_elements):
        b[ind4e[i]] += J[i]*np.matmul(M_R, f(c4n[ind4e[i]]))
    row_ind = np.tile(ind4e.flatten(),(k+1,1)).T.flatten()
    col_ind = np.tile(ind4e,(1,k+1)).flatten()
    A_COO = coo_matrix((Aloc.flatten(), (row_ind, col_ind)), shape=(number_of_nodes, number_of_nodes))
    A = A_COO.tocsr()
    dof = np.setdiff1d(range(0,number_of_nodes), n4db)
    u[dof] = spsolve(A[dof, :].tocsc()[:, dof].tocsr(), b[dof])
    return u

In [ ]:
def compute_error_fem_1d(c4n,ind4e,M_R,D_R,u,Du):
    error = 0
    number_of_elements = len(ind4e)
    J = np.array([(c4n[ind4e[i,-1]]-c4n[ind4e[i,0]])/2 for i in range(number_of_elements)])
    for i in range(number_of_elements):
        u_x = np.matmul(D_R, u[ind4e[i]]) / J[i]
        D_e = Du(c4n[ind4e[i]]) - u_x
        error += J[i] * np.matmul(D_e,np.matmul(M_R,D_e))
    return np.sqrt(error)

In [ ]:
iter = 10
a = 0
b = 1
k = 3
M = 2 ** np.arange(2,iter+2)
f = lambda x: 25 * np.pi**2 * np.sin(5 * np.pi * x)
u_D = lambda x: 0 * x
Du = lambda x: 5 * np.pi * np.cos(5 * np.pi * x)

error = np.zeros(iter)
h = 1 / M

In [ ]:
M_R, S_R, D_R = get_matrices_1d(k)
for i in range(iter):
    c4n, n4e, n4db, ind4e = mesh_fem_1d(a,b,M[i],k)
    u = fem_for_poisson_1d(c4n,n4e,n4db,ind4e,k,M_R,S_R,f,u_D)
    error[i] = compute_error_fem_1d(c4n,ind4e,M_R,D_R,u,Du)

rate = (np.log(error[1:])-np.log(error[:-1]))/(np.log(h[1:])-np.log(h[:-1]))

In [ ]:
print(rate)

**[Exercise 2]** Modify **fem_for_poisson_1d_ex2** to solve the Poisson problem with non-homogeneous Dirichlet boundary condition,
\begin{align*}
		-u''(x) &= \ f(x) \qquad \textrm{in } \Omega \\
		u(x) &= u_D(x) \quad \ \ \ \textrm{on } \partial\Omega. \\
\end{align*}

In [ ]:
def fem_for_poisson_1d_ex2(c4n,n4e,n4db,ind4e,k,M_R,S_R,f,u_D):
	number_of_nodes = len(c4n)
	number_of_elements = len(n4e)
	b = np.zeros(number_of_nodes)
	u = np.zeros(number_of_nodes)
	J = np.array([(c4n[n4e[i,1]]-c4n[n4e[i,0]])/2 for i in range(number_of_elements)])
	Aloc = np.array([S_R.flatten()/J[i] for i in range(number_of_elements)])
	for i in range(number_of_elements):
		b[ind4e[i]] += J[i]*np.matmul(M_R, f(c4n[ind4e[i]]))
	row_ind = np.tile(ind4e.flatten(),(k+1,1)).T.flatten()
	col_ind = np.tile(ind4e,(1,k+1)).flatten()
	A_COO = coo_matrix((Aloc.flatten(), (row_ind, col_ind)), shape=(number_of_nodes, number_of_nodes))
	A = A_COO.tocsr()
	dof = np.setdiff1d(range(0,number_of_nodes), n4db)

	###################################################################################################
	# TODO: Add a few lines to treat non-homogeneous Dirichlet boundary condition.
	
	###################################################################################################

	u[dof] = spsolve(A[dof, :].tocsc()[:, dof].tocsr(), b[dof])
	return u

In [ ]:
iter = 10
a = 0
b = 1
k = 2
M = 2 ** np.arange(2,iter+2)
f = lambda x: 25 * np.pi**2 * np.sin(5 * np.pi * x)
u_D = lambda x: x
Du = lambda x: 5 * np.pi * np.cos(5 * np.pi * x) + 1

error = np.zeros(iter)
h = 1 / M

M_R, S_R, D_R = get_matrices_1d(k)
for i in range(iter):
	c4n, n4e, n4db, ind4e = mesh_fem_1d(a,b,M[i],k)
	u = fem_for_poisson_1d_ex2(c4n,n4e,n4db,ind4e,k,M_R,S_R,f,u_D)
	error[i] = compute_error_fem_1d(c4n,ind4e,M_R,D_R,u,Du)

rate = (np.log(error[1:])-np.log(error[:-1]))/(np.log(h[1:])-np.log(h[:-1]))
print(rate)

**[Exercise 3]** Modify **fem_for_poisson_1d_ex3** to solve the Poisson problem with mixed boundary condition,
\begin{align*}
		-u''(x) &= \ f(x) \qquad \textrm{in } \Omega \\
		u(x) &= u_D(x) \qquad \textrm{on } \Gamma_D \\
		u'(x)\boldsymbol{n} &= u_N(x) \qquad \textrm{on } \Gamma_N,
\end{align*}
where $\Gamma_D$ denotes the Dirichlet boundary, $\Gamma_N$ denotes the Neumann boundary, and $\boldsymbol{n}$ is the outward unit normal vector.

In [ ]:
def fem_for_poisson_1d_ex3(c4n,n4e,n4db,n4nb,ind4e,k,M_R,S_R,f,u_D,u_N):
	number_of_nodes = len(c4n)
	number_of_elements = len(n4e)
	b = np.zeros(number_of_nodes)
	u = np.zeros(number_of_nodes)
	J = np.array([(c4n[n4e[i,1]]-c4n[n4e[i,0]])/2 for i in range(number_of_elements)])
	Aloc = np.array([S_R.flatten()/J[i] for i in range(number_of_elements)])
	for i in range(number_of_elements):
		b[ind4e[i]] += J[i]*np.matmul(M_R, f(c4n[ind4e[i]]))
	row_ind = np.tile(ind4e.flatten(),(k+1,1)).T.flatten()
	col_ind = np.tile(ind4e,(1,k+1)).flatten()
	A_COO = coo_matrix((Aloc.flatten(), (row_ind, col_ind)), shape=(number_of_nodes, number_of_nodes))
	A = A_COO.tocsr()
	dof = np.setdiff1d(range(0,number_of_nodes), n4db)

	###################################################################################################
	# TODO: Add a few lines to treat Neumann boundary condition.
	
	###################################################################################################

	###################################################################################################
	# TODO: Add a few lines to treat non-homogeneous Dirichlet boundary condition. 
	#       If you finished Exercise 2, the following lines can be copied from `fem_for_poisson_1d_ex2`
	
	###################################################################################################

	u[dof] = spsolve(A[dof, :].tocsc()[:, dof].tocsr(), b[dof])
	return u

In [ ]:
iter = 10
a = 0
b = 1
k = 1
M = 2 ** np.arange(2,iter+2)
f = lambda x: 25 * np.pi**2 * np.sin(5 * np.pi * x)
u_D = lambda x: x
u_N = lambda x: -5 * np.pi * np.cos(5 * np.pi * x) - 1
Du = lambda x: 5 * np.pi * np.cos(5 * np.pi * x) + 1

error = np.zeros(iter)
h = 1 / M

M_R, S_R, D_R = get_matrices_1d(k)
for i in range(iter):
	c4n, n4e, n4db, ind4e = mesh_fem_1d(a,b,M[i],k)
	n4nb = n4db[0]
	n4db = n4db[1]
	u = fem_for_poisson_1d_ex3(c4n,n4e,n4db,n4nb,ind4e,k,M_R,S_R,f,u_D,u_N)
	error[i] = compute_error_fem_1d(c4n,ind4e,M_R,D_R,u,Du)

rate = (np.log(error[1:])-np.log(error[:-1]))/(np.log(h[1:])-np.log(h[:-1]))
print(rate)